In [1]:
import pandas as pd
import numpy as np
from nltk.stem.porter import *

In [2]:
data = pd.read_csv('data_all.csv')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
vectorizer = TfidfVectorizer()
stemmer = PorterStemmer()
texts = data['text'].str.replace('[^А-я]', ' ', regex=True).str.lower()
texts = [' '.join([stemmer.stem(word) for word in text.split()]) for text in texts]

In [5]:
train_vectors = vectorizer.fit_transform(texts)

In [6]:
!pip install lightgbm
import lightgbm
import xgboost
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

In [7]:
target = np.array(data.drop(['text', 'Unnamed: 52', 'total'], axis=1))
target_np = np.argmax(target, axis=1)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(train_vectors, target, test_size=0.2)

In [31]:
n_estimators = 300
learning_rate = 1
model = xgboost.XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)
model_default = xgboost.XGBClassifier()
y_train_default = np.argmax(y_train, axis=1) == 0
model.fit(X_train, y_train)
model_default.fit(X_train, y_train_default)
preds = model.predict(X_test)
preds_default = model_default.predict(X_test)
preds_result = np.argmax(preds, axis=1) == 0
true_result = np.argmax(y_test, axis=1) == 0
print(accuracy_score(np.argmax(preds, axis=1), np.argmax(y_test,axis=1)), recall_score(preds_result, true_result))

0.4088200238379023 0.5300988319856245


In [32]:
texts_test = pd.Series([
    'тестовое предложение'
])
texts_test = texts_test.str.replace('[^А-я]', ' ', regex=True).str.lower()
texts_test = [' '.join([stemmer.stem(word) for word in text.split()]) for text in texts_test]

In [33]:
test_vectors = vectorizer.transform(texts_test)
predictions_test = model.predict(test_vectors)
preds_test = np.argmax(predictions_test, axis=1)
preds_default = model_default.predict(test_vectors)
print(list(map(lambda x:data.drop(['text', 'Unnamed: 52', 'total'], axis=1).columns[x],preds_test)))
print(preds_default)

['👍']
[0]


In [35]:
model_file = open('models/XGB_model_vectors', 'wb')
model_default_file = open('models/XGB_model_vectors_default', 'wb')
vectorizer_file = open('models/vectorizer', 'wb')
import pickle
pickle.dump(model, model_file)
pickle.dump(model_default, model_default_file)
pickle.dump(vectorizer, vectorizer_file)
model_file.close()
model_default_file.close()
vectorizer_file.close()

In [17]:
np.argmax(y_train, axis=1).shape

(6711,)